In [1]:
import Gpib
import time
import matplotlib
import matplotlib.pyplot as plt

import ipywidgets as widgets

import plotly.plotly as py
import plotly.figure_factory as ff
import plotly.graph_objs as go

from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

import numpy as np

%matplotlib inline
import smu as sm
import meas


In [2]:

inst = Gpib.Gpib(0, 2)
smu = sm.hp4142b(inst)
m=meas.pmosfet(smu=smu)
m.idss(Vgs=-4, Vds=-0.5, Ids_max=1)
print("Vgs_on = %.2f V" %m.vgs_on(Ids=1e-3))


HEWLETT PACKARD,4142B,0,4.30

0,0,0,0

IBSTA: Complete END
0,0,0,0

IBSTA: Complete END
Vgs_on = -3.39 V


In [3]:
#Measure n JFET operating curves
data = []

results = []


#for vgs in np.arange(-3, -7, -1):
for vgs in [-4, -6, -8, -10]:

    d=m.ids_vds(Vgs=float(vgs), Vds_max=-5, Ids_max=1, P_max=1)
    print(vgs)
    ids=np.array(d['I_D'])
    igs=np.array(d['I_G'])
    vds=np.array(d['V_D'])
    results.append ( { 'Ids': ids,
                        'Igs': igs,
                        'Vds': vds,
                        'Vgs': vgs,
                      'part': '2N7000',
                     't': time.time() } )

    
print("READY.")


0,0,0,0

IBSTA: Complete END
-4
227,0,0,0

IBSTA: Complete END
MEAS_ERR: CBI-1.00002E+00
MEAS_ERR: TDI-0.28610E-09
MEAS_ERR: VBI+199.999E+99
-6
227,0,0,0

IBSTA: Complete END
MEAS_ERR: CBI-1.00000E+00
MEAS_ERR: TDI-0.56774E-09
MEAS_ERR: VBI+199.999E+99
-8
227,0,0,0

IBSTA: Complete END
MEAS_ERR: CBI-1.00004E+00
MEAS_ERR: VBI+199.999E+99
-10
READY.


In [4]:
data= []

for r in results:
    trace =  go.Scatter(x=r['Vds'], y=1e3*r['Ids'], name='Vgs=%.2f' % (r['Vgs']))
    data.append(trace)

layout = go.Layout(
    title='D8P05',
    xaxis=dict(
        title='V_ds(V)',
        autorange='reversed',

        #type='log',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#000000'
        )
    ),
    yaxis=dict(
        title='I_ds(mA)',
        autorange='reversed',

        #range=[0, 300],
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#000000'
        )
    )
)


iplot({ 'data': data, "layout": layout })

In [5]:
ids_vs_vgs = m.ids_vgs(Ids_max=1)

trace =  go.Scatter(x=ids_vs_vgs['V_G'], y=ids_vs_vgs['I_D'])

layout = go.Layout(
    title='D8P05',
    xaxis=dict(
        title='V_gs(V)',
        autorange='reversed',
    ),
    yaxis=dict(
        title='I_ds(A)',
        autorange='reversed',
    )
)

iplot({ 'data': [trace], "layout": layout })

227,0,0,0

IBSTA: Complete END
MEAS_ERR: CBI-1.00004E+00
MEAS_ERR: TDI-0.44248E-09
MEAS_ERR: VBI+199.999E+99
